In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
os.environ['USE_PYGEOS'] = '0'

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics
from calitp_data_analysis import get_fs, geography_utils
from shared_utils import rt_utils, rt_dates

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

ModuleNotFoundError: No module named 'shared_utils'

In [ ]:
import segment_speed_utils

In [ ]:
analysis_date = rt_dates.DATES['sep2023']

In [4]:
fs = get_fs()

In [5]:
path = 'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/avg_speeds_stop_segments_2023-09-13.parquet'

In [6]:
fs.get(path, './avg_speeds_stop_segments_2023-09-13.parquet')

[None]

In [41]:
gdf = gpd.read_parquet(path).to_crs(geography_utils.CA_NAD83Albers)

In [42]:
gdf >> head(3)

,shape_array_key,stop_sequence,schedule_gtfs_dataset_key,stop_id,loop_or_inlining,geometry,district_name,p50_mph,n_trips,p20_mph,p80_mph,time_of_day
0,00116752596dddc4b59481bed1ce62be,840,84d3c3507a4c8df851d935b63908bbd5,4044,0,"LINESTRING (179705.832 -444552.051, 179668.180...",District 7 - Los Angeles,17.56,1,17.56,17.56,all_day
1,00116752596dddc4b59481bed1ce62be,960,84d3c3507a4c8df851d935b63908bbd5,4052,0,"LINESTRING (173246.127 -442114.715, 173246.127...",District 7 - Los Angeles,18.22,1,18.22,18.22,all_day
2,00116752596dddc4b59481bed1ce62be,1200,84d3c3507a4c8df851d935b63908bbd5,4068,0,"LINESTRING (172375.056 -442007.187, 172247.238...",District 7 - Los Angeles,15.10,1,15.10,15.10,all_day


In [43]:
rt_utils.get_speedmaps_ix_df?

Signature:
rt_utils.get_speedmaps_ix_df(
    analysis_date: datetime.date,
    itp_id: Optional[int] = None,
) -> pandas.core.frame.DataFrame
Docstring:
Collect relevant keys for finding all schedule and rt data for a reports-assessed organization.
Note that organizations may have multiple sets of feeds, or share feeds with other orgs.

Used with specific itp_id in rt_analysis.rt_parser.OperatorDayAnalysis, or without specifying itp_id
to get an overall table of which datasets were processed and how to deduplicate if needed
File:      ~/data-analyses/_shared_utils/shared_utils/rt_utils.py
Type:      function

In [44]:
samo = rt_utils.get_speedmaps_ix_df(analysis_date) >> filter(_.organization_name.str.contains('Santa Monica')) >> head(10)

In [45]:
segments_samo = (gdf >> filter(_.time_of_day == 'all_day', _.schedule_gtfs_dataset_key.isin(samo.schedule_gtfs_dataset_key))).copy()

In [46]:
segments_samo >> head(3)

,shape_array_key,stop_sequence,schedule_gtfs_dataset_key,stop_id,loop_or_inlining,geometry,district_name,p50_mph,n_trips,p20_mph,p80_mph,time_of_day
2733,02d3552393ac04db7f4adccd79cbd019,2,efbbd5293be71f7a5de0cf82b59febe1,1126,0,"LINESTRING (139241.450 -443495.408, 139250.645...",District 7 - Los Angeles,7.98,1,7.98,7.98,all_day
2734,02d3552393ac04db7f4adccd79cbd019,3,efbbd5293be71f7a5de0cf82b59febe1,286,0,"LINESTRING (139591.223 -443837.363, 139601.611...",District 7 - Los Angeles,16.76,1,16.76,16.76,all_day
2735,02d3552393ac04db7f4adccd79cbd019,4,efbbd5293be71f7a5de0cf82b59febe1,1666,0,"LINESTRING (139702.986 -444005.741, 139690.139...",District 7 - Los Angeles,11.75,1,11.75,11.75,all_day


In [47]:
segments_samo.geometry = segments_samo.geometry.apply(rt_utils.try_parallel)

In [48]:
segments_samo = segments_samo.apply(rt_utils.arrowize_by_frequency, axis=1, args=['n_trips'])

In [49]:
segments_samo >> head(3)

,shape_array_key,stop_sequence,schedule_gtfs_dataset_key,stop_id,loop_or_inlining,geometry,district_name,p50_mph,n_trips,p20_mph,p80_mph,time_of_day
2733,02d3552393ac04db7f4adccd79cbd019,2,efbbd5293be71f7a5de0cf82b59febe1,1126,0,"POLYGON ((139210.510 -443528.088, 139233.062 -...",District 7 - Los Angeles,7.98,1,7.98,7.98,all_day
2734,02d3552393ac04db7f4adccd79cbd019,3,efbbd5293be71f7a5de0cf82b59febe1,286,0,"POLYGON ((139560.045 -443869.871, 139582.498 -...",District 7 - Los Angeles,16.76,1,16.76,16.76,all_day
2735,02d3552393ac04db7f4adccd79cbd019,4,efbbd5293be71f7a5de0cf82b59febe1,1666,0,"POLYGON ((139497.765 -444186.178, 139575.026 -...",District 7 - Los Angeles,11.75,1,11.75,11.75,all_day


In [50]:
segments_samo.crs

<Projected CRS: EPSG:3310>
Name: NAD83 / California Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - California.
- bounds: (-124.45, 32.53, -114.12, 42.01)
Coordinate Operation:
- name: California Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [51]:
import shapely

## Segs to spa util

In [53]:
## TODO ignore centroid warning

In [60]:
rt_utils.set_state_export(segments_samo, cmap=rt_utils.ZERO_THIRTY_COLORSCALE, legend_url=rt_utils.SPEEDMAP_LEGEND_URL,
                                      map_type='speedmap',
                          color_col='p20_mph', filename='samo_segments_test2')

/home/jovyan/data-analyses/_shared_utils/shared_utils/rt_utils.py:877: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


writing to calitp-map-tiles/testing/samo_segments_test2.geojson.gz


{'state_dict': {'name': 'null',
  'layers': [{'name': 'Map',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/samo_segments_test2.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.02783664626179, -118.44884642755206),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3Rpbmcvc2Ftb19zZWdtZW50c190ZXN0Mi5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjUsICJ0b29sdGlwX3NwZWVkX2tleSI6ICJwMjBfbXBoIn0sICJ0eXBlIjogInNwZWVkbWFwIn1dLCAibGF0X2xvbiI6IFszNC4wMjc4MzY2NDYyNjE3OSwgLTExOC40NDg4NDY0Mjc1NTIwNl0sICJ6b29tIjogMTMsICJsZWdlbmRfdXJsIjogImh0dHBzOi8vc3Rvc

## Speedmap Comparision

In [ ]:
from rt_analysis import rt_filter_map_plot

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(300, analysis_date)

In [ ]:
_m = rt_day.segment_speed_map(no_render=True)

In [ ]:
rt_day.map_gz_export()

In [ ]:
rt_day.spa_map_state

In [ ]:
rt_day.spa_map_url

In [ ]:
## TODO shn?

In [ ]:
rt_day.display_spa_map()